In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv("trader.csv")

In [20]:
df.columns = df.columns.str.strip() #Removing spaces in the column names

In [21]:
df['oper_date_AMPM'] = df['oper_date'].str[-2:] #Separating am/pm into separate column

In [22]:
df["oper_date"] = pd.to_datetime(pd.Series(df["oper_date"]), format='%d-%b-%y %I.%M.%S.%f %p') #Converting into datetime and 24 hrs format
df["oper_date"] = df["oper_date"].apply(lambda x: x.replace(microsecond=0)) # Removing microseconds from the string

In [23]:
df = df.sort_values(by=['account_id','CP','oper_date'], ascending=[True, True, True]) #Sorting the values
df["cashflow"] = -df["amount"] * df["rate"] # Calculating cashflow
df["year"] = df['oper_date'].astype(str).str[0:4] # Creating Year column from oper_date column

In [24]:
df

,account_id,creation_date,CP,oper_date,resulted_rate,resulted_amount,resulted_weight,rate,amount,weight,oper_date_AMPM,cashflow,year
2902,598192,09-JUN-10 05.06.20.148 PM,AUD/CAD,2014-01-06 09:33:18,0.95443,-10000.0,-8946.90,0.95443,-10000.0,-8946.90,AM,9544.30,2014
2906,598192,09-JUN-10 05.06.20.148 PM,AUD/CAD,2014-01-06 11:21:50,0.95443,0.0,0.00,0.95551,10000.0,8952.60,AM,-9555.10,2014
3042,598192,09-JUN-10 05.06.20.148 PM,AUD/CAD,2014-04-09 22:22:04,1.02549,1000.0,941.71,1.02549,1000.0,941.71,PM,-1025.49,2014
3062,598192,09-JUN-10 05.06.20.148 PM,AUD/CAD,2014-04-10 00:49:50,1.02549,0.0,0.00,1.02505,-1000.0,-940.73,AM,1025.05,2014
3662,598192,09-JUN-10 05.06.20.148 PM,AUD/CAD,2020-03-06 05:53:08,0.89067,10000.0,6640.95,0.89067,10000.0,6640.95,AM,-8906.70,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45444,848277,01-FEB-12 03.34.33.942 AM,XAU/USD,2022-07-26 08:21:39,1717.14000,0.0,0.00,1717.16000,2.0,3433.67,AM,-3434.32,2022
45457,848277,01-FEB-12 03.34.33.942 AM,XAU/USD,2022-07-26 09:32:13,1721.98000,1.0,1721.80,1721.98000,1.0,1721.80,AM,-1721.98,2022
45458,848277,01-FEB-12 03.34.33.942 AM,XAU/USD,2022-07-26 09:36:38,1721.98000,0.0,0.00,1720.57000,-1.0,-1720.78,AM,1720.57,2022
45461,848277,01-FEB-12 03.34.33.942 AM,XAU/USD,2022-07-26 10:21:05,1721.53000,1.0,1721.33,1721.53000,1.0,1721.33,AM,-1721.53,2022


In [8]:
unique_accid = df["account_id"].unique().tolist() #Getting the unique account id's

In [9]:
unique_accid
df = df[["account_id", "CP", "oper_date", "resulted_amount", "cashflow", "year"]].reset_index() # Filtering selected columns
df = df[["account_id", "CP", "oper_date", "resulted_amount", "cashflow", "year"]] # Removing index column

In [10]:
df.head()

,account_id,CP,oper_date,resulted_amount,cashflow,year
0,598192,AUD/CAD,2014-01-06 09:33:18,-10000.0,9544.30,2014
1,598192,AUD/CAD,2014-01-06 11:21:50,0.0,-9555.10,2014
2,598192,AUD/CAD,2014-04-09 22:22:04,1000.0,-1025.49,2014
3,598192,AUD/CAD,2014-04-10 00:49:50,0.0,1025.05,2014
4,598192,AUD/CAD,2020-03-06 05:53:08,10000.0,-8906.70,2020


In [11]:
full_list = []
for i in unique_accid:
    temp_acc_df = df[df["account_id"] == i]
    unique_cp = temp_acc_df["CP"].unique().tolist()
    for j in unique_cp:
        temp_cp_df = temp_acc_df[temp_acc_df["CP"] ==j].reset_index()
        #print(temp_cp_df.head())
        ctr = 0
        for k, row in temp_cp_df.iterrows():
            finder = 0
            if temp_cp_df["resulted_amount"][k] == 0.0:
                finder = k
                temp_df_fil = temp_cp_df[ctr:finder+1]
                profit = temp_df_fil["cashflow"].sum()
                if profit>0:
                    win = 1
                    loss = 0
                else:
                    win = 0
                    loss = 1
                ctr = finder+1
                full_list.append([row["account_id"], row["CP"], row["oper_date"],row["resulted_amount"],row["cashflow"],row["year"], profit, win, loss])

In [12]:
cols = ["account_id", "CP", "oper_date", "resulted_amount", "cashflow", "year", "profit_loss", "win", "loss"]
fin_df = pd.DataFrame(full_list, columns= cols)

In [13]:
fin_df

,account_id,CP,oper_date,resulted_amount,cashflow,year,profit_loss,win,loss
0,598192,AUD/CAD,2014-01-06 11:21:50,0.0,-9555.10,2014,-10.80,0,1
1,598192,AUD/CAD,2014-04-10 00:49:50,0.0,1025.05,2014,-0.44,0,1
2,598192,AUD/CAD,2020-03-08 18:02:20,0.0,8979.10,2020,72.40,1,0
3,598192,AUD/CAD,2020-03-10 01:08:42,0.0,8934.90,2020,-19.20,0,1
4,598192,AUD/CAD,2020-03-11 15:13:05,0.0,8932.50,2020,-22.20,0,1
...,...,...,...,...,...,...,...,...,...
15914,848277,XAU/USD,2022-07-26 04:09:26,0.0,1724.77,2022,-1.18,0,1
15915,848277,XAU/USD,2022-07-26 08:06:57,0.0,1717.28,2022,-1.64,0,1
15916,848277,XAU/USD,2022-07-26 08:21:39,0.0,-3434.32,2022,-1.67,0,1
15917,848277,XAU/USD,2022-07-26 09:36:38,0.0,1720.57,2022,-1.41,0,1


In [14]:
final = (fin_df.groupby(['account_id','year', 'CP']).agg(win=('win', 'sum'), loss=('loss', 'sum')).reset_index())

In [15]:
final["win_loss_ratio"] = round((final["win"] / (final["win"] + final["loss"]))*100,2)

In [16]:
final

,account_id,year,CP,win,loss,win_loss_ratio
0,598192,2010,AUD/JPY,2,8,20.00
1,598192,2010,AUD/NZD,1,0,100.00
2,598192,2010,AUD/USD,4,4,50.00
3,598192,2010,CAD/JPY,0,2,0.00
4,598192,2010,CHF/JPY,7,7,50.00
...,...,...,...,...,...,...
305,848277,2022,USD/CHF,13,25,34.21
306,848277,2022,USD/CNH,3,2,60.00
307,848277,2022,USD/JPY,218,301,42.00
308,848277,2022,USD/SGD,1,6,14.29


In [17]:
final.to_csv("Output.csv",index=None)